## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [ ]:
import pandas as pd

df = pd.read_csv('/datasets/data.csv')
df.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [ ]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [ ]:
df.duplicated().sum()

54

In [ ]:
df.describe() 

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Вывод

В предоставленном датасете находятся 21525 записи о клиентах банка (из которых 54 - полные дубликаты).
В колонках "days_employed" и "total_income" по 2174 пропущенных значений (около 10%).
Из полученных значений статистических характеристик видно, что в исходных данных содержится информация о людях до 75 лет с разным уровнем образования, рабочего стажа. Ежемесячный доход клиентов в выборке варьируется в диапазоне от 20667 до 2265604.
Сразу можно заметить необычные, скорее всего ошибочные, данные:
1) Отрицательное значение числа детей 

2) Как отрицательные, так и положительные значения общего трудового стажа в днях.

3) Нулевое значение возраста клиента в годах.



### Шаг 2. Предобработка данных

### Обработка пропусков

Исследуем природу пропущенных значений в столбцах "days_employed" и "total_income".
Для этого поищем записи в которых значение в одной колонке пропущено, а в другой нет.

In [ ]:
df['total_income'][df['days_employed'].isnull()].notnull().sum()


0

Видим, что значения пропущены в одних и тех же строках.
Возможно, это означает, что у людей, которые еще не работали ни одного дня, нет и дохода (NaN стоит вместо 0).
Для проверки этой гипотезы, посмотрим что еще известно об этих людях. 

In [ ]:
print('age, amount')
df['dob_years'][df['days_employed'].isnull()].value_counts().head(10)

age, amount


34    69
40    66
42    65
31    65
35    64
36    63
47    59
41    59
30    58
28    57
Name: dob_years, dtype: int64

In [ ]:
df['income_type'][df['days_employed'].isnull()].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Так как это дотаточно взрослые люди с определенным типом занятости, можно предположить, что они на самом деле имеют трудовой стаж, а их значения "days_employed" и "total_income" были пропущены или потеряны. Заменить их нулями будет некорректно. Удалить строки с NaN тоже нельзя, потому что таким образом мы потеряем около 10% данных в других столбцах.

Посмотрим на колонку "days_employed"

In [ ]:
df['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

In [ ]:
df_grouped = df.groupby('income_type').agg({'days_employed': ['min', 'max', 'median']})
df_grouped

days_employed                              
                           min            max         median
income_type                                                 
безработный      337524.466835  395302.838654  366413.652744
в декрете         -3296.759962   -3296.759962   -3296.759962
госслужащий      -15193.032201     -39.954170   -2689.368353
компаньон        -17615.563266     -30.195337   -1547.382223
пенсионер        328728.720605  401755.400475  365213.306266
предприниматель    -520.848083    -520.848083    -520.848083
сотрудник        -18388.949901     -24.141633   -1574.202821
студент            -578.751554    -578.751554    -578.751554

Значения в этой колонке варьируются от -18388 до 401755. 

Рассмотрим область положительных значений 'days_employed'

In [ ]:
print('Клиенты с положительными значениями "days_employed" по роду занятия :\n',
      df['income_type'][df['days_employed'] >= 0].unique())

Клиенты с положительными значениями "days_employed" по роду занятия :
 ['пенсионер' 'безработный']


In [ ]:
print('Самое крупное положительное значение стажа :', df['days_employed'][(df['days_employed'] > 0)].max(), 'дней')
print('Что соответствует', df['days_employed'][(df['days_employed'] > 0)].max() / 365, "лет")

Самое крупное положительное значение стажа : 401755.40047533 дней
Что соответствует 1100.6997273296713 лет


In [ ]:
print('Самое маленькое положительное значение стажа :', df['days_employed'][(df['days_employed'] > 0)].min(), 'дней')
print('Что соответствует', df['days_employed'][(df['days_employed'] > 0)].min() / 365, "лет")

Самое маленькое положительное значение стажа : 328728.72060451825 дней
Что соответствует 900.6266317932007 лет


Мы видим, что положительные значения 'days_employed' встречаются только у пенсионеров и безработных.
При этом положительные значения столбца 'days_employed' соответствуют нереальным значениям стажа от 900 до 1100 лет.
Нужно получить уточнение у кредитного отдела банка об этих данных.
Рассмотрим область отрицательных значений 'days_employed'



In [ ]:
print('Клиенты с отрицательными значениями "days_employed" по роду занятия :\n',
      df['income_type'][df['days_employed'] < 0].value_counts())

Клиенты с отрицательными значениями "days_employed" по роду занятия :
 сотрудник          10014
компаньон           4577
госслужащий         1312
студент                1
предприниматель        1
в декрете              1
Name: income_type, dtype: int64


In [ ]:
print('Самое крупное отрицательное значение стажа :', df['days_employed'][(df['days_employed'] <= 0)].min(), 'дней')
print('Что соответствует', df['days_employed'][(df['days_employed'] <= 0)].min() / 365, "лет")

Самое крупное отрицательное значение стажа : -18388.949900568383 дней
Что соответствует -50.38068465909146 лет


In [ ]:
print('Самое маленькое отрицательное значение стажа :', df['days_employed'][(df['days_employed'] <= 0)].max(), 'дней')
print('Что соответствует', df['days_employed'][(df['days_employed'] <= 0)].max() / 365, "года")

Самое маленькое отрицательное значение стажа : -24.14163324048118 дней
Что соответствует -0.06614146093282515 года


Значения трудового стажа от 0,07 до 50,4 лет вполне соответствуют ожиданиям.



Заменим NaN в столбце 'days_employed' для группы пенсионеров и безработных на медианное значение положительной части столбца 'days_employed' и для группы остальных клиентов на медианное значение отрицательной части

In [ ]:
pos_employed_median = df['days_employed'][df['days_employed'] > 0].median()
df['days_employed'][(df['income_type'].isin(['пенсионер', 'безработный'])) & (df['days_employed'].isna())] = pos_employed_median

neg_employed_median = df['days_employed'][df['days_employed'] <= 0].median()
df['days_employed'][(df['income_type'].isin(['сотрудник', 'компаньон', 'госслужащий', 'в декрете', 'студент', 'предприниматель'])) & (df['days_employed'].isna())] = neg_employed_median


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Сдеаем то же самое для столбца 'total_income'


In [ ]:
pos_income_median = df['total_income'][df['days_employed'] > 0].median()
df['total_income'][(df['income_type'].isin(['пенсионер', 'безработный'])) & (df['total_income'].isna())] = pos_income_median

neg_income_median = df['total_income'][df['days_employed'] <= 0].median()
df['total_income'][(df['income_type'].isin(['сотрудник', 'компаньон', 'госслужащий', 'в декрете', 'студент', 'предприниматель'])) & (df['total_income'].isna())] = neg_income_median


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
df['days_employed'] = df['days_employed'].abs()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Пропущенные значения в столбцах "days_employed" и "total_income" - случайные пропуски, которые не зависят от значений в других столбцах. Мы заменили их на медианные значения сответствующих величин, разделив датасет на две категории по типу занятости: 

1) Пенсионеры и безработные

2) Остальные типы занятости

Медианные значения подходят лучше, чем средние, потому что они не чувствительны к единичным выбросам значения величины.



### Замена типа данных

В датасете есть два столбца с типом данных float64 (вещественные числа) - это уже исследованные нами "days_employed" и "total_income". В обоих случаях нам не нужна высокая точность до нескольких знаков после запятой, достаточно целых значений числа рабочих дней и сумм ежемесячного дохода "без копеек".

In [ ]:
try:
    df['days_employed'] = df['days_employed'].round().astype('int64')
    df['total_income'] = df['total_income'].round().astype('int64')
except:
    print('Ошибка формата')
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8438,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилья
1,1,4025,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилья
3,3,4125,32,среднее,1,женат / замужем,0,M,сотрудник,0,267629,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [ ]:
df.dtypes

children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

### Вывод

Оптимальный метод замены типа значений float на int - astype, но он округляет дробную часть числа до нуля вниз. Чтобы точнее перевести значения в целочисленные, мы можем сначала округлить вещественные числа методом round.
В таблице остались только строковые и целочисленные значения



### Обработка дубликатов

In [ ]:
df.duplicated().sum()

54

In [ ]:
df = df.drop_duplicates().reset_index(drop = True)
df.shape

(21471, 12)

После удаления 54 полных дубликатов в датасете осталось 21471 строки.
Теперь нужно найти частичные дубликаты вручную.
Изучим столбцы со строковыми значениями.

In [ ]:
df['education'].value_counts()

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Проверим, влияет ли вариант написания на присвоенное значение 'education_id'

In [ ]:
df.groupby('education_id')['education'].unique()

education_id
0                             [высшее, ВЫСШЕЕ, Высшее]
1                          [среднее, Среднее, СРЕДНЕЕ]
2    [неоконченное высшее, НЕОКОНЧЕННОЕ ВЫСШЕЕ, Нео...
3                    [начальное, НАЧАЛЬНОЕ, Начальное]
4     [Ученая степень, УЧЕНАЯ СТЕПЕНЬ, ученая степень]
Name: education, dtype: object

Идентификаторы уровня образования присвоены правильно. 
Переведем все символы в столбце 'education' в нижний регистр

In [ ]:
df['education'] = df['education'].str.lower()
df['education'].value_counts()

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [ ]:
df.duplicated().sum()

17

Мы обнаружили еще 17 дубликатов, различавшихся стилем написания уровня образования клиента.
Уберем эти дубликаты

In [ ]:
df = df.drop_duplicates().reset_index(drop = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Проверим остальные столбцы со строчными даннными

In [ ]:
print('Уникальные значения family_status :',df['family_status'].unique())
print()
print('Уникальные значения gender :', df['gender'].unique())
print()
print('Уникальные значения income_type :', df['income_type'].unique())
print()
print('Уникальные значения purpose :', df['purpose'].unique())

Уникальные значения family_status : ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']

Уникальные значения gender : ['F' 'M' 'XNA']

Уникальные значения income_type : ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']

Уникальные значения purpose : ['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 '

In [ ]:
df['family_status'] = df['family_status'].str.lower()

### Вывод

Мы нашли и удалили 54 полных дубликата и еще 17 дубликатов с разным видом написания уровня образования в столбце 'education' . 
Для дальнейшего анализа нужно провести лемматизацию значений в столбце 'purpose'



### Лемматизация

Найдем какие слова встречаются в столбце 'purpose'. 

In [ ]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem() 
word_list = []
for value in df['purpose'].unique():
    lemmas = m.lemmatize(value)
    word_list += lemmas
        
print(Counter(word_list))

Counter({' ': 59, '\n': 38, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'подержать': 1, 'со': 1, 'подержанный': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1})


Мы можем составить список ключевых слов по которым будут однозначно определены цели кредитов. 

In [ ]:
purpose_list = ['недвижимость', 'автомобиль', 'образование', 'жилье', 'свадьба']

In [ ]:

def purpose_choose(row):
    purpose = row['purpose']
    lemmas = m.lemmatize(purpose)
    for purpose_type in purpose_list:
        if (purpose_type in lemmas):
            return purpose_type



df['purpose_type'] = df.apply(purpose_choose, axis=1) 
df['purpose_type'].value_counts()

недвижимость    6351
жилье           4460
автомобиль      4306
образование     4013
свадьба         2324
Name: purpose_type, dtype: int64

Проверим датасет на дубликаты без изначального столбца 'purpose'.

In [ ]:
df = df.drop(['purpose'], axis = 1)
df.duplicated().sum()

244

In [ ]:
df = df.drop_duplicates().reset_index(drop = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21210 entries, 0 to 21209
Data columns (total 12 columns):
children            21210 non-null int64
days_employed       21210 non-null int64
dob_years           21210 non-null int64
education           21210 non-null object
education_id        21210 non-null int64
family_status       21210 non-null object
family_status_id    21210 non-null int64
gender              21210 non-null object
income_type         21210 non-null object
debt                21210 non-null int64
total_income        21210 non-null int64
purpose_type        21210 non-null object
dtypes: int64(7), object(5)
memory usage: 1.9+ MB


### Вывод

Все уникальные значения столбца 'purpose' мы разделили на 5 категорий. 
Благодаря этому мы смогли найти и удалить еще 244 дубликата, различавшихся только формулировкой цели кредита. 
Вcего нам удалось найти в иходном датасете 315 дубликатов.

Скорее всего, причина возникновения дубликатов состоит в копировании данных из нескольких баз с разными форматами написания категорий и целей кредита.



### Категоризация данных

Для ответа на поставленные вопросы нам необходимо провести категоризацию данных в соответствующих столбцах.
Начнем со столбца 'children'.  

In [ ]:
df['children'].value_counts()

 0     13892
 1      4773
 2      2043
 3       329
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [ ]:
df.groupby('children').mean()

,days_employed,dob_years,education_id,family_status_id,debt,total_income
children,,,,,,
-1,63145.191489,42.574468,0.829787,0.957447,0.021277,152995.000000
0,92218.663691,46.174201,0.829758,1.130507,0.076447,163281.528002
1,22836.083805,38.375864,0.786298,0.812696,0.093023,169267.536769
2,5665.122859,35.764562,0.790015,0.452766,0.094958,169857.335781
3,8518.352584,36.264438,0.823708,0.404255,0.082067,179848.167173
4,11774.390244,36.048780,0.780488,0.512195,0.097561,167244.731707
5,1454.222222,38.777778,1.222222,0.222222,0.000000,168286.222222
20,44692.328947,41.815789,0.842105,0.815789,0.105263,165779.657895


Видим 47 записей со значением 'children' равным -1. По остальным числовым параметрам в среднем эти клиенты наиболее близки к людям без детей. Правда, среднее значение 'debt' в этой группе очень низко, что может быть обусловлено малым размером выборки.
Заменим значения 'children' c -1 на 0.

In [ ]:
df['children'] = df['children'].replace(-1, 0)

Разделим клиентов банка на 4 группы, у кого 0, 1, 2 и более детей. Статус сохраним в новом столбце 'children_status'

In [ ]:
def children_status(children):

        if children == 0:
                return 0
        if children == 1:
                return 1
        if children == 2:
                return 2
        return 3
    
df['children_status'] = df['children'].apply(children_status)

Добавим столбец с категоризацией клиентов по их доходу так, чтобы количество человек в группах было сопоставимо.

In [ ]:
def income_group(income):

        if income < 100000:
                return 'низкий доход'
        if income <= 200000:
                return 'средний доход'
        return 'высокий доход'
    
df['income_level'] = df['total_income'].apply(income_group)
df.groupby('income_level')['children'].count()

income_level
высокий доход     5066
низкий доход      4463
средний доход    11681
Name: children, dtype: int64

Для оптимизации датасета выделим словари с категориями образования и семейного положения

In [ ]:
df_log = df[['days_employed', 'dob_years', 'education_id',
       'family_status_id', 'gender', 'income_type', 'debt',
       'purpose_type', 'children_status', 'income_level']]

df_family_dict = df[['family_status', 'family_status_id']]
df_family_dict = df_family_dict.drop_duplicates().reset_index(drop = True)

df_education_dict = df[['education', 'education_id']]
df_education_dict = df_education_dict.drop_duplicates().reset_index(drop = True)


### Вывод

Для классификации датасета по параметрам с числовыми значениями мы разделили значения столбца 'children' на четыре категории, а столбца 'total_income' на три.  
Обращение по идентификаторам 'family_status_id' и 'education_id' упрощает работу с таблицей



### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
def df_group(data, column):  # создает итоговую таблицу
    data_grouped = data.groupby(column).agg({'debt': ['sum', 'count']})
    data_grouped['percent'] = data_grouped['debt']['sum'] / data_grouped['debt']['count'] * 100
    return data_grouped

In [ ]:
df_children_grouped = df_group(df, 'children_status')
df_children_grouped

debt          percent
                  sum  count          
children_status                       
0                1063  13939  7.626085
1                 444   4773  9.302326
2                 194   2043  9.495839
3                  39    455  8.571429

In [ ]:
r = df_children_grouped[df_children_grouped.index != 0].sum()['debt']['sum'] / df_children_grouped[df_children_grouped.index != 0].sum()['debt']['count']
print(f'Процент клиентов с детьми, имеющих задолженности : {r:.2%}')

Процент клиентов с детьми, имеющих задолженности : 9.31%


### Вывод

Как мы видим, только у 7,62% людей без детей есть задолженности по кредитам, в то время, как у людей с детьми - в 9,31% случаев.
Возможно, это связано с более тяжелым материальным положением людей с детьми.
При этом какой-то зависимости в количестве должников между группами с одним, двумя и более детей не наблюдается.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
df_family_grouped = df_group(df, 'family_status')
df_family_grouped.sort_values(by = 'percent',ascending = False)

debt          percent
                       sum  count          
family_status                              
не женат / не замужем  274   2792  9.813754
гражданский брак       388   4129  9.396948
женат / замужем        930  12148  7.655581
в разводе               85   1193  7.124895
вдовец / вдова          63    948  6.645570

In [ ]:
df_family_age_grouped = df.groupby('family_status')['dob_years'].mean().sort_values(ascending = True)
print('Средний возраст по категориям ', df_family_age_grouped)

Средний возраст по категориям  family_status
не женат / не замужем    38.380014
гражданский брак         42.024946
женат / замужем          43.476457
в разводе                45.503772
вдовец / вдова           56.443038
Name: dob_years, dtype: float64


### Вывод

Наибольший процент должников у незамужних людей, что более характерно для молодых людей (38,4 года). Наиболее надежные клиенты в категории "вдовец / вдова" со средним возрастом 56,4 года. 
Остальные данные по возврату кредитов в срок также кореллируют со средним возрастом клиентов в категории 'family_status'.
Стоит отметить значительную разницу в поведении клиентов, состоящих в официальном браке и гражданском. Это говорит о более ответственном отношении к финансам людей, оформивших брак официально.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
df_income_grouped = df_group(df, 'income_level')

df_income_grouped.sort_values(by = 'percent',ascending = False)

debt          percent
                sum  count          
income_level                        
средний доход  1028  11681  8.800616
низкий доход    354   4463  7.931884
высокий доход   358   5066  7.066719

### Вывод

Никакой явной зависимости наличия просроченных платежей от уровня дохода нет. 

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
df_purpose_grouped = df_group(df, 'purpose_type')

df_purpose_grouped.sort_values(by = 'percent',ascending = False)

debt         percent
              sum count          
purpose_type                     
автомобиль    402  4272  9.410112
образование   370  3964  9.334006
свадьба       186  2306  8.065915
недвижимость  474  6260  7.571885
жилье         308  4408  6.987296

### Вывод

Клиенты чаще пропускают платежи по кредитам на автомобили или образование, чем на приобретение недвижимости. Это, долно быть, связано с осознанием важности цели кредита и более тчательной проверкой на платежеспособность со стороны банков. Отличие результатов по группам "недвижимость" и "жилье" заметное, но однозначно определить цель приобретения недвижимости (для проживания или коммерческого использования) по имеющимся данным невозможно.



### Шаг 4. Общий вывод

Мы провели классификацию данных о клиентах банков и определили ряд закономерностей в поведении разных групп людей.

* Клиенты с детьми чаще задерживают платежи по кредитам
* Семейный статус клиента влияет на вероятность возврата им кредита в срок. 
* По указанной цели кредита можно определить риск невозврата.
* Уровень дохода клиента не определяет его пунктуальность в платежах.

Процент людей с просроченными платежами по различным группам варьируется от 6,64% до 9,81%, т.е. примерно на треть. Более тчательное изучение заявителей на получение кредита может значительно увеличить вероятность своевременного погашения долга.

